In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import tensorflow as tf
import csv

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [68]:
from collections import defaultdict


artist_csv = pd.read_csv('../data/artists.csv')
#remove artists with multiple genres
artist_csv = artist_csv[~artist_csv["genre"].str.contains(",")].reset_index()

#find number of total paintings per genre
dic = {}
for g in artist_csv["genre"].unique():
   dic[g] = artist_csv[artist_csv["genre"] == g]["paintings"].sum()

dic
# #keep only name and genre cols in df
# artist_csv = artist_csv[["name", "genre"]]
# #replace spaces with underscores
# artist_csv["name"] = artist_csv["name"].str.replace(" ", "_")


{'Expressionism': 193,
 'Impressionism': 1370,
 'Surrealism': 241,
 'Byzantine Art': 99,
 'Post-Impressionism': 1048,
 'Northern Renaissance': 680,
 'Suprematism': 126,
 'Symbolism': 171,
 'Cubism': 439,
 'Baroque': 586,
 'Romanticism': 388,
 'Mannerism': 87,
 'Primitivism': 309,
 'Proto Renaissance': 119,
 'Early Renaissance': 164,
 'High Renaissance': 301,
 'Realism': 59,
 'Neoplasticism': 84,
 'Pop Art': 181,
 'Abstract Expressionism': 24}

In [3]:
artist_csv["genre"] = artist_csv["genre"].str.replace(" ", "_")
genre_names = artist_csv["genre"].unique().tolist()
print(genre_names)

['Expressionism', 'Impressionism', 'Surrealism', 'Byzantine_Art', 'Post-Impressionism', 'Northern_Renaissance', 'Suprematism', 'Symbolism', 'Cubism', 'Baroque', 'Romanticism', 'Mannerism', 'Primitivism', 'Proto_Renaissance', 'Early_Renaissance', 'High_Renaissance', 'Realism', 'Neoplasticism', 'Pop_Art', 'Abstract_Expressionism']


### Agumentation

In [78]:
#hyperparameters
batch_size = 16
train_input_shape = (224, 224, 3)
n_classes = len(genre_names)

img_dir = '../data/genre_images/'

train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255., zoom_range=0.7)

train_generator = train_datagen.flow_from_directory(directory=img_dir,
                                                    class_mode='categorical',
                                                    classes=genre_names,
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset='training',
                                                    shuffle=True,)

valid_generator = train_datagen.flow_from_directory(directory=img_dir,
                                                    class_mode='categorical',
                                                    classes=genre_names,
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset='validation',
                                                    shuffle=True,)

                                                    
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)                                           


Found 3983 images belonging to 20 classes.
Found 984 images belonging to 20 classes.
Total number of batches = 248 and 61


In [79]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=train_input_shape)

for layer in base_model.layers:
    layer.trainable = False

In [80]:
# Add layers at the end
X = base_model.output
X = Flatten()(X)

X = Dense(512, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(16, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

output = Dense(n_classes, activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=output)



In [81]:
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

In [84]:
n_epoch = 8

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              verbose=1, mode='auto')


In [85]:
# Train the model
model.fit(train_generator, 
            steps_per_epoch=STEP_SIZE_TRAIN, 
            epochs=n_epoch, 
            batch_size=batch_size,
            validation_data=valid_generator, 
            validation_steps=STEP_SIZE_VALID, 
            callbacks=[early_stop, reduce_lr], 
            verbose=1)

Epoch 1/8
248/248 [==============================] - 242s 961ms/step - loss: 2.7468 - accuracy: 0.1961 - val_loss: 3.3447 - val_accuracy: 0.0840 - lr: 1.0000e-04
Epoch 2/8
248/248 [==============================] - 204s 821ms/step - loss: 2.6454 - accuracy: 0.2312 - val_loss: 3.0561 - val_accuracy: 0.1199 - lr: 1.0000e-04
Epoch 3/8
248/248 [==============================] - 202s 815ms/step - loss: 2.5634 - accuracy: 0.2657 - val_loss: 3.0050 - val_accuracy: 0.1762 - lr: 1.0000e-04
Epoch 4/8
248/248 [==============================] - 202s 816ms/step - loss: 2.5167 - accuracy: 0.2763 - val_loss: 2.8890 - val_accuracy: 0.2152 - lr: 1.0000e-04
Epoch 5/8
248/248 [==============================] - 201s 812ms/step - loss: 2.4449 - accuracy: 0.2972 - val_loss: 3.2629 - val_accuracy: 0.1824 - lr: 1.0000e-04
Epoch 6/8
248/248 [==============================] - 200s 805ms/step - loss: 2.4107 - accuracy: 0.3065 - val_loss: 2.7872 - val_accuracy: 0.1793 - lr: 1.0000e-04
Epoch 7/8
248/248 [=========